## Trabalho 3 - Laboratório de Otimização

### Questão 1: (1.0)

Considere a implementação do VRTW aprensetada em sala de aula
https://github.com/jesusossian/opt_lab_2024_1/blob/main/notebooks/vrp_vrptw.ipynb

Considere ainda a tabela instância-aluno.
| alunos | nC | mNV | vC | dR | tH | tWWR | sTR | mT |
|:----|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
| CARLOS DANIEL |[30] | [4,5,6] | [30,40] | [(1,4)] | [100] |[(10,15)] | [(1,3)] | [1,2] |
| DANIEL BRUNO | [30] | [4,5,6] | [30,40] | [(1,4)] | [100] | [(15,20)] | [(1,3)] | [1,2] |
| DAVI PEREIRA | [30] | [4,5,6] | [30,40] | [(1,4)] | [100] | [(20,25)] | [(1,3)] | [1,2] |
| FABRICIO ARAUJO | [30] | [4,5,6] | [30,40] | [(1,4)] | [100] | [(25,30)] | [(1,3)] | [1,2] | 
| GABRIEL DIMITRI | [30] | [4,5,6] | [30,40] | [(2,5)] | [100] | [(30,35)] | [(2,4)] | [1,2] |
| ICARO DE SOUSA | [30] | [4,5,6] | [30,40] | [(2,5)] | [100] | [(35,40)] | [(2,4)] | [1,2] |
| LUCELENA DOS SANTOS | [30] | [4,5,6] | [30,40] | [(2,5)] | [100] | [(40,45)] | [(2,4)] | [1,2] |

1. Execute as instância alocada a sua pessoa de acordo com a tabela instância-aluno. Configure os parametros **nC, mNV, vC, dR, tH, tWWR, sTR** e **mT** de acordo com a tabela.

2. Crie um dataframe com os resultados obtidos.

3. Implemente uma função que gere o gráfico das rotas associada a solução ótima do problema. Execute essa função para pelo menos uma instância.

### Questão 2: (2.0)

1. Descreva o ambiente computacional utilizado.

2. Faça um relatório baseado nos resultados encontrados e suas observações. Considere as informações contidas no dataframe criado e o retorno da função **create_route()**.

3. Enviar os arquivos gerados pelos experimentos computacionais.

### Import library

In [19]:
# library
import gurobipy as gp
from gurobipy import GRB
import math
import random
import pandas as pd

### Instance Generation

In [20]:
# Instance Generation

def instances_generation(numCustomers, maxNumVehicles, demandRange, timeHorizon, timeWindowWidthRange, serviceTimeRange):

    random.seed(0)

    depot = 0
    customers = [*range(1, numCustomers + 1)]
    locations = [depot] + customers
    connections = [(i, j) for i in locations for j in locations if i != j]
    #vehicles = [*range(1, maxNumVehicles + 1)]

    # create random depot and customer locations in the Euclidian plane (1000x1000)
    points = [(random.randint(0, 999), random.randint(0, 999)) for i in locations]

    # dictionary of Euclidean distance for each connection (interpreted as travel costs)
    costs = {
        (i, j): math.ceil(
            math.sqrt(sum((points[i][k] - points[j][k]) ** 2 for k in range(2)))
        )
        for (i, j) in connections
    }
    maximalCosts = math.ceil(999 * math.sqrt(2))

    # dictionary of travel times for each connection (related to the costs, scaled to time horizon)
    travelTimes = {
        (i, j): math.ceil((costs[i, j] / maximalCosts) * timeHorizon * 0.2)
        for (i, j) in connections
    }

    # create random demands, service times, and time window widths in the given range
    demands = {i: random.randint(demandRange[0], demandRange[1]) for i in customers}
    demands[0] = 0  # depot has no demand
    serviceTimes = {
        i: random.randint(serviceTimeRange[0], serviceTimeRange[1]) for i in customers
    }
    serviceTimes[0] = 0  # depot has no service time
    timeWindowWidths = {
        i: random.randint(timeWindowWidthRange[0], timeWindowWidthRange[1])
        for i in customers
    }

    # vehicles are allowed to leave the depot any time within the time horizon
    timeWindowWidths[0] = timeHorizon

    # create time windows randomly based on the previously generated information
    # such that the service at a customer can be finished within the time horizon
    timeWindows = {}
    timeWindows[0] = (0, 0 + timeWindowWidths[0])
    for i in customers:
        start = random.randint(0, timeHorizon - serviceTimes[i] - timeWindowWidths[i] - travelTimes[i,0])
        timeWindows[i] = (start, start + timeWindowWidths[i])

    return demands, connections, costs, customers, locations, timeWindows, serviceTimes, travelTimes

In [21]:
# number of vehicles needed for customers
def numVehiclesNeededForCustomers(customers, demands, vehicleCapacity):
    sumDemand = 0
    
    for i in customers:
        sumDemand += demands[i]

    return math.ceil(sumDemand / vehicleCapacity)

In [22]:
# VRPTW formulation
def basic_form(connections, costs, customers, maxNumVehicles, demands, vehicleCapacity):

    # create model for VRPTW instance
    model = gp.Model("VRPTW")

    # binary variables x(i,j): is 1 if some vehicle is going from node i to node j, 0 otherwise
    x = model.addVars(connections, vtype=GRB.BINARY, name="x")

    # objective function: minimize sum of connection costs
    model.setObjective(x.prod(costs), GRB.MINIMIZE)

    # all customers have exactly one incoming and one outgoing connection
    model.addConstrs((x.sum("*", j) == 1 for j in customers), name="incoming")
    model.addConstrs((x.sum(i, "*") == 1 for i in customers), name="outgoing")

    # vehicle limits
    model.addConstr(x.sum(0, "*") <= maxNumVehicles, name="maxNumVehicles")
    model.addConstr(
        x.sum(0, "*") >= numVehiclesNeededForCustomers(customers, demands, vehicleCapacity),
        name="minNumVehicles",
    )

    # salve data         
    model._data = x

    model.update()

    return model

### Implementation of Load and Time Models: big-M or flow

In [23]:
# add constraints by big M
def addLoadConstraintsByBigM(model, locations, vehicleCapacity, demands, customers):

    x = model._data
    y = model.addVars(locations, lb=0, ub=vehicleCapacity, name="y")
    y[0].UB = 0  # empty load at depot

    model.addConstrs(
        (
            y[i] + demands[j] <= y[j] + vehicleCapacity * (1 - x[i, j])
            for i in locations
            for j in customers
            if i != j
        ),
        name="loadBigM1",
    )
    model.addConstrs(
        (
            y[i] + demands[j]
            >= y[j] - (vehicleCapacity - demands[i] - demands[j]) * (1 - x[i, j])
            for i in locations
            for j in customers
            if i != j
        ),
        name="loadBigM2",
    )

    model.update()

In [24]:
# add constraints by flows
def addLoadConstraintsByFlows(model, connections, vehicleCapacity, customers, demands, locations):

    x = model._data

    z = model.addVars(connections, lb=0, ub=vehicleCapacity, name="z")

    for i in customers:
        z[0, i].UB = 0

    model.addConstrs(
        (z.sum("*", j) + demands[j] == z.sum(j, "*") for j in customers),
        name="flowConservation",
    )
    model.addConstrs(
        (
            z[i, j] >= demands[i] * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="loadLowerBound",
    )
    model.addConstrs(
        (
            z[i, j] <= (vehicleCapacity - demands[j]) * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="loadUpperBound",
    )

    model.update()

In [25]:
# add time constraints by big M
def addTimeConstraintsByBigM(model, locations, timeWindows, serviceTimes, travelTimes, customers):

    x = model._data
    
    y = model.addVars(locations, name="y")
    for i in locations:
        y[i].LB = timeWindows[i][0]
        y[i].UB = timeWindows[i][1]

    model.addConstrs(
        (
            y[i] + serviceTimes[i] + travelTimes[i, j]
            <= y[j]
            + (
                timeWindows[i][1]
                + serviceTimes[i]
                + travelTimes[i, j]
                - timeWindows[j][0]
            )
            * (1 - x[i, j])
            for i in locations
            for j in customers
            if i != j
        ),
        name="timeBigM",
    )

    model.update()

In [26]:
# add time constraints by flows
def addTimeConstraintsByFlows(model, connections, timeWindows, serviceTimes, travelTimes, locations, customers):

    x = model._data
    
    z = model.addVars(connections, lb=0, name="z")

    for (i, j) in connections:
        z[i, j].UB = timeWindows[i][1]

    model.addConstrs(
        (
            gp.quicksum(
                z[i, j] + (serviceTimes[i] + travelTimes[i, j]) * x[i, j]
                for i in locations
                if (i, j) in connections
            )
            <= z.sum(j, "*")
            for j in customers
        ),
        name="flowConservation",
    )
    model.addConstrs(
        (
            z[i, j] >= timeWindows[i][0] * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="timeWindowStart",
    )
    model.addConstrs(
        (
            z[i, j] <= timeWindows[i][1] * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="timeWindowEnd",
    )

    model.update()

In [27]:
def load_time_model(model, locations, connections, vehicleCapacity, demands, customers, timeWindows, serviceTimes, travelTimes, modelType):
    
    ### model configuration
    #loadModelType = 1  # 1: big-M, 2: flow
    #timeModelType = 1  # 1: big-M, 2: flow

    if modelType == 1:
        (loadModelType,timeModelType) = (1,1)
    else:
        (loadModelType,timeModelType) = (2,2)

    if loadModelType == 1:
        addLoadConstraintsByBigM(model, locations, vehicleCapacity, demands, customers)
    elif loadModelType == 2:
        addLoadConstraintsByFlows(model, connections, vehicleCapacity, customers, demands, locations)

    if timeModelType == 1:
        addTimeConstraintsByBigM(model, locations, timeWindows, serviceTimes, travelTimes, customers)
    elif timeModelType == 2:
        addTimeConstraintsByFlows(model, connections, timeWindows, serviceTimes, travelTimes, locations, customers)

### Solve model

In [28]:
def solve(model):

    # parameters value
    MAX_CPU_TIME = 3600
    EPSILON= 1.e-6

    # export .lp
    #model.write(file_name+"_model.lp")

    # parameters config
    model.Params.TimeLimit = MAX_CPU_TIME # time limite
    model.Params.MIPGap = EPSILON # MIPGap
    model.Params.method = 1 # method root
    model.Params.NodeMethod = 1 #  -1=automatic, 0=primal simplex, 1=dual simplex, and 2=barrier
    model.Params.Threads = 1
    model.Params.OutputFlag = 0
    
    # Turn off display
    #gp.setParam('OutputFlag', 0)

    model.update()

    model.optimize()

    status = 0
    if model.status == GRB.OPTIMAL:
        status = 1
 
    bound = model.objBound
    val = model.objVal
    gap = model.MIPGap
    time = model.Runtime
    node = model.NodeCount

    return status, bound, val, gap, time, node

In [29]:
# create opt router
def create_route(model, numCustomers, travelTimes, demands, timeWindows, serviceTimes, vehicleCapacity, customers):
    
    x = model._data

    if model.SolCount >= 1:

        usedConnections = [(i, j) for (i, j) in x.keys() if x[i, j].X > 0.5]

        # create a dict for the next customer based on the current one
        # (note that the depot in general has multiple outgoing connections)
        nextCustomer = {}
        for (i, j) in usedConnections:
            if i == 0:
                if 0 not in nextCustomer.keys():
                    nextCustomer[0] = []
                nextCustomer[0].append(j)
            else:
                nextCustomer[i] = j

        print(f"Solution contains {len(nextCustomer[0])} routes:")
        routeNumber = 0
        visitedCustomers = [False] * (numCustomers + 1)
        for firstCustomer in nextCustomer[0]:
            print(f"Route {routeNumber}: 0 -> ", end="")
            vehicleLoad = 0
            time = travelTimes[0, firstCustomer]
            violatedTimeWindows = False
            currentCustomer = firstCustomer
            while currentCustomer != 0:
                print(f"{currentCustomer} (L:{vehicleLoad}, T:{time}) -> ", end="")
                visitedCustomers[currentCustomer] = True
                vehicleLoad += demands[currentCustomer]
                time = max(time, timeWindows[currentCustomer][0])
                if time > timeWindows[currentCustomer][1]:
                    violatedTimeWindows = True
                time += (
                    serviceTimes[currentCustomer]
                    + travelTimes[currentCustomer, nextCustomer[currentCustomer]]
                )
                currentCustomer = nextCustomer[currentCustomer]
            print(f"0 (L:{vehicleLoad}/{vehicleCapacity}, T:{time})")
            if vehicleLoad > vehicleCapacity:
                print("Vehicle capacity is exceeded!")
            if violatedTimeWindows:
                print("Time windows are violated!")
            routeNumber += 1

        print("Unvisited customers: ", end="")
        for c in customers:
            if visitedCustomers[c] == False:
                print(f"{c}, ", end="")
        print(" ")

In [30]:
# vrp main
def vrp(nC, mNV, vC, dR, tH, tWWR, sTR, modelType):

    numCustomers = nC
    maxNumVehicles = mNV
    vehicleCapacity = vC
    demandRange = dR
    timeHorizon = tH
    timeWindowWidthRange = tWWR
    serviceTimeRange = sTR
   
    demands, connections, costs, customers, locations, timeWindows, serviceTimes, travelTimes =  \
    instances_generation(
        numCustomers, maxNumVehicles, demandRange, timeHorizon, timeWindowWidthRange, serviceTimeRange
        )

    model = basic_form(connections, costs, customers, mNV, demands, vehicleCapacity)

    load_time_model(
        model, locations, connections, vehicleCapacity, demands, \
        customers, timeWindows, serviceTimes, travelTimes, modelType
        )
    
    status, bound, val, gap, time, node = solve(model)

    create_route(
        model, numCustomers, travelTimes, demands, timeWindows, serviceTimes, vehicleCapacity, customers
        )

    # free resources
    model.dispose()

    return status, bound, val, gap, time, node

In [31]:
if __name__ == "__main__":

    # numCustomers: nC \in [30]
    # maxNumVehicles: mNV \in [4,5,6]
    # vehicleCapacity: vC \in [30,40]
    # demandRange: dR \in [(1,4),(2,5)]
    # timeHorizon: tH \in [100]
    # timeWindowWidthRange: tWWR \in [(10,15),(15,20),(20,25),(25,30),(30,35),(35,40)]
    # serviceTimeRange: sTR \in [(1,3),(2,4)]
    # modelType: mT \in [1,2] : 1(big-M), 2(flow)

    for nC in [30]:
        for mNV in [4,5]:
            for vC in [40]:
                for dR in [(2,5)]:
                    for tH in [100]:
                        for tWWR in [(10,15)]:
                            for sTR in [(1,3)]:
                                for mT in [1,2]:
                                    print(f"nC{nC}_mNV{mNV}_vC{vC}_dR{dR}_tH{tH}_tWWR{tWWR}_sTR{sTR}_mT{mT}")
                                    status, bound, val, gap, time, node = vrp(nC, mNV, vC, dR, tH, tWWR, sTR, mT)

nC30_mNV4_vC40_dR(2, 5)_tH100_tWWR(10, 15)_sTR(1, 3)_mT1
Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 1e-06
Set parameter Method to value 1
Set parameter NodeMethod to value 1
Set parameter Threads to value 1
Solution contains 4 routes:
Route 0: 0 -> 1 (L:0, T:8) -> 5 (L:4, T:16) -> 9 (L:9, T:20) -> 15 (L:12, T:28) -> 3 (L:15, T:32) -> 7 (L:17, T:34) -> 14 (L:22, T:75) -> 6 (L:25, T:79) -> 21 (L:27, T:82) -> 0 (L:29/40, T:92)
Route 1: 0 -> 20 (L:0, T:5) -> 10 (L:5, T:12) -> 27 (L:9, T:16) -> 18 (L:13, T:30) -> 13 (L:15, T:51) -> 0 (L:18/40, T:83)
Route 2: 0 -> 28 (L:0, T:4) -> 17 (L:5, T:11) -> 22 (L:7, T:17) -> 25 (L:11, T:24) -> 4 (L:15, T:30) -> 23 (L:17, T:34) -> 16 (L:21, T:39) -> 26 (L:26, T:53) -> 30 (L:29, T:68) -> 0 (L:34/40, T:74)
Route 3: 0 -> 29 (L:0, T:6) -> 11 (L:2, T:12) -> 2 (L:4, T:19) -> 19 (L:6, T:24) -> 12 (L:10, T:26) -> 24 (L:13, T:33) -> 8 (L:15, T:42) -> 0 (L:19/40, T:84)
Unvisited customers:  
nC30_mNV4_vC40_dR(2, 5)_tH100_tWWR(10, 15)_sT